# SLOPE for Count Data

*Autors: Zachary Lau, Joey Hotz and Javier Martinez-Rodriguez*

## Introduction (280 words)

This project aims to measure the performance of Sorted L-One Penalized Estimation (SLOPE) for variable selection in the context of high-dimensional count data modeled with Poisson regression. SLOPE extends the LASSO penalty by introducing a sequence of non-increasing regularization parameters, resulting in the penalty $\sum_{i=1}^{p}\lambda_{i}|\hat{\beta}_{(i)}|$, where the coefficients $|\hat{\beta}|$ are sorted in descending order and $\lambda_1 \ge \dots \ge \lambda_p \ge 0$  (Bogdan et al., 2015). While SLOPE, inspired by the Benjamini-Hochberg (BH) procedure, has shown theoretical False Discovery Rate (FDR) control in Gaussian linear models, its behaviour in non-Gaussian settings, such as Poisson regression, remains less explored. For this reason, this project aims to determine how the performance of SLOPE regarding variable selection accuracy (FDR and Power) compares to LASSO and Adaptive LASSO when applied to high-dimensional Poisson regression.

This project uses simulations to compare the variable selection accuracy (measured by FDR and Power) of SLOPE against standard and adaptive LASSO across scenarios varying in predictor dimensionality ($p/n$ ratio), inter-predictor correlation ($\rho$), sparsity ($k$), and signal strength, using a target FDR ($q=0.1$) for SLOPE and cross-validation for LASSO variants.

To discuss the details of the SLOPE penalization and the experiments evaluating its performance using simulation data, this document is divided into X parts. First, it provides a general explanation of SLOPE applied to Gaussian linear models and the effect of choosing different procedures to assign its values, as discussed by Bogdan et al. (2015). In particular, it discusses the FDR provided by the Benjamini-Hochberg procedure. Second, the implementation of SLOPE in Generalized Linear Models (GLM) is addressed, particularly with count data, as proposed by Larsson et al. (2024). Third, the simulation settings, competing penalizations, and results are discussed. 


## Explanation of SLOPE

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## Explanation of SLOPE for Count Data

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## Experiments

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

### Competing Penalizations

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

### Experimental Settings (212 words)

To compare the variable selection performance of SLOPE against LASSO and Adaptive LASSO for high-dimensional data, count data is generated following a Poisson distribution with a log-linear link given by $\log(\lambda_i) = \beta_0 + X_i \beta$. The comparison is measured in terms of FDR and Power across 50 replications to ensure statistical stability. Performance is assessed using SLOPE tuned to a target FDR of $q=0.1$ via a BH-inspired $\lambda$ sequence using 10-fold cross-validation, while LASSO and Adaptive LASSO tune $\lambda$ using 10-fold cross-validation minimizing Poisson deviance.

The chosen experimental settings are conditions to challenge variable selection. The dimensionality cases include scenarios where predictors outnumber observations, are equal and are fewer ($p \in \{500, 1000, 2000\}$ with $n=1000$). Predictor correlation is evaluated with none, medium, and high correlation ($\rho \in \{0, 0.5, 0.8\}$). This is particularly relevant since correlation among regressors introduces complications in the selection of variables.

The underlying sparsity level ($k \in \{10, 20, 50, 100\}$ active predictors) is evaluated at different levels to see how methods cope as the number of true signals increases relative to the total predictors. Finally, simulating both "Weak" and "Strong" signal strengths (magnitudes of non-zero $\beta_j$) tests the sensitivity (Power) of each method to detect subtle versus obvious effects. The code below implements the generative model for the discussed settings. 

In [ ]:
# Setup ----
## Packages to use ----
if (!require("pacman")) install.packages("pacman")
# if (!require("mytidyfunctions")) remotes::install_github("JavierMtzRdz/mytidyfunctions")

pacman::p_load(tidyverse, janitor, 
               SLOPE, glmnet, MASS,
              # mytidyfunctions,
               progress,
               patchwork, here)

## Load fonts ----
extrafont::loadfonts(quiet = TRUE)

## Set theme ------
# mytidyfunctions::set_mytheme(text = element_text(family = "Times New Roman"))


In [12]:
# Generative models
## Simulation Parameters
n <- 1000         
p_values <- c(500, 1000, 2000) 
rho_values <- c(0, 0.5, 0.8)  
k_values <- c(10, 20, 50, 100) # Non-zero betas
signal_strengths <- list( 
  weak = list(beta_min = 0.1, beta_max = 0.5),
  strong = list(beta_min = 0.5, beta_max = 1.5)
)
R <- 1 
q_fdr <- 0.1       # q parameter for SLOPE
adapt_lasso_gamma <- 1 # ALasso weights
beta0 <- 0.5

set.seed(538)

## Generative model
generate_data <- function(n, p, rho, k, signal_info, beta0) {

  beta_true <- numeric(p)

  if (k > 0) {
    non_zero_indices <- sample(1:p, k)
    magnitudes <- runif(k, min = signal_info$beta_min, max = signal_info$beta_max)
    signs <- sample(c(-1, 1), k, replace = TRUE)
    beta_true[non_zero_indices] <- magnitudes * signs
  }
  true_support <- which(beta_true != 0)

  # Generate X
  Sigma <- matrix(rho, nrow = p, ncol = p)
  diag(Sigma) <- 1
  X <- MASS::mvrnorm(n = n, mu = rep(0, p), Sigma = Sigma)
  X <- scale(X)

  # Count response 
  lambda <- exp(beta0 + X %*% beta_true)
  # lambda <- pmin(lambda, some_large_value)
  y <- rpois(n, lambda)

  return(list(X = X, y = y, beta_true = beta_true, true_support = true_support, beta0 = beta0))
}

In [ ]:
# Function to calculate FDR and Power
calculate_metrics <- function(selected_indices, true_indices, p) {
  true_positives <- length(intersect(selected_indices, true_indices))
  false_positives <- length(setdiff(selected_indices, true_indices))
  # Power
  power <- ifelse(length(true_indices) == 0, NA, true_positives / length(true_indices))
  # FDR
  fdr <- ifelse((true_positives + false_positives) == 0, 0, false_positives / (true_positives + false_positives))
  return(list(FDR = fdr, Power = power, TP = true_positives, FP = false_positives, Selected_Count = length(selected_indices)))
}


# Loop ---

results_list <- list()
total_runs <- length(p_values) * length(rho_values) * length(k_values) * length(signal_strengths) * R
cli::cli_progress_bar("Cleaning data", total = total_runs)
iter <- 0

for (p in p_values) {
  for (rho in rho_values) {
    for (k in k_values) {
      for (signal_name in names(signal_strengths)) {
        signal_info <- signal_strengths[[signal_name]]

        for (rep in 1:R) {
          cli::cli_progress_update()
          iter <- iter + 1

          # Generate Data
          sim_data <- generate_data(n = n, p = p, rho = rho, k = k,
                                    signal_info = signal_info, beta0 = beta0)
          X <- sim_data$X
          y <- sim_data$y
          true_support <- sim_data$true_support # Indices of non-zero elements in beta_true

          # --- Fit Models ---

          # SLOPE
          selected_slope <- integer(0) # Initialize as empty
          fdr_slope <- NA
          power_slope <- NA
          slope_error <- FALSE
          slope_fit <- SLOPE::SLOPE(X, y, family = "poisson", fdr = q_fdr, lambda = "bh")
          slope_coeffs <- coef(slope_fit)

          selected_slope <- which(slope_coeffs[-1] != 0)
          metrics_slope <- calculate_metrics(selected_slope, true_support, p)
          fdr_slope <- metrics_slope$FDR
          power_slope <- metrics_slope$Power


          # LASSO 
          selected_lasso <- integer(0)
          fdr_lasso <- NA
          power_lasso <- NA
          lasso_error <- FALSE
          cv_lasso_fit <- cv.glmnet(X, y, family = "poisson", alpha = 1, standardize = FALSE)
            lasso_coeffs <- coef(cv_lasso_fit, s = "lambda.min")
            selected_lasso <- which(lasso_coeffs[-1] != 0) 
              metrics_lasso <- calculate_metrics(selected_lasso, true_support, p)
              fdr_lasso <- metrics_lasso$FDR
              power_lasso <- metrics_lasso$Power


          # Adaptive LASSO
          selected_adapt <- integer(0)
          fdr_adapt <- NA
          power_adapt <- NA
          adapt_error <- FALSE

            cv_ridge_fit <- cv.glmnet(X, y, family = "poisson", alpha = 0, standardize = FALSE) 
            ridge_coeffs <- coef(cv_ridge_fit, s = "lambda.min")[-1] 

            # Calculate weights 
            weights <- 1 / (abs(ridge_coeffs) + .Machine$double.eps)^adapt_lasso_gamma
            weights <- pmin(weights, 1e10) 

            # Fit weighted LASSO using CV
            cv_adapt_fit <- cv.glmnet(X, y, family = "poisson", alpha = 1,
                                      penalty.factor = weights, standardize = FALSE)
            adapt_coeffs <- coef(cv_adapt_fit, s = "lambda.min")
            selected_adapt <- which(adapt_coeffs[-1] != 0) 
      
            metrics_adapt <- calculate_metrics(selected_adapt, true_support, p)
            fdr_adapt <- metrics_adapt$FDR
            power_adapt <- metrics_adapt$Power

          # Save results
          results_list[[run_counter]] <- data.frame(
            n = n, p = p, rho = rho, k = k, signal = signal_name, replication = rep,
            Method = c("SLOPE", "LASSO", "AdaptiveLASSO"),
            FDR = c(fdr_slope, fdr_lasso, fdr_adapt),
            Power = c(power_slope, power_lasso, power_adapt),
            SelectedCount = c(length(selected_slope), length(selected_lasso), length(selected_adapt))
          )

        } 
      } 
    } 
  } 
} 
cli::cli_progress_done()

final_results <- bind_rows(results_list)

There were 50 or more warnings (use warnings() to see the first 50)


In [22]:
coef(cv_adapt_fit)

2001 x 1 sparse Matrix of class "dgCMatrix"
                  s1
(Intercept) 14.15154
V1           .      
V2           .      
V3           .      
V4           .      
V5           .      
V6           .      
V7           .      
V8           .      
V9           .      
V10          .      
V11          .      
V12          .      
V13          .      
V14          .      
V15          .      
V16          .      
V17          .      
V18          .      
V19          .      
V20          .      
V21          .      
V22          .      
V23          .      
V24          .      
V25          .      
V26          .      
V27          .      
V28          .      
V29          .      
V30          .      
V31          .      
V32          .      
V33          .      
V34          .      
V35          .      
V36          .      
V37          .      
V38          .      
V39          .      
V40          .      
V41          .      
V42          .      
V43          .      
V44        

### Results

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## References

Bogdan, Małgorzata, Ewout van den Berg, Chiara Sabatti, Weijie Su, and Emmanuel J. Candès. 2015. “Slope—Adaptive Variable Selection Via Convex Optimization.” The Annals of Applied Statistics 9 (3): 1103–40.

Larsson, Johan, Jonas Wallin, Malgorzata Bogdan, Ewout van den Berg, Chiara Sabatti, Emmanuel Candes, Evan Patterson, et al. 2024. “SLOPE: Sorted L1 Penalized Estimation.” R. CRAN. https://cran.r-project.org/web/packages/SLOPE/index.html.


